In [37]:
from Bio import SeqIO, Entrez
import os
from urllib.error import HTTPError
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from itertools import permutations, product
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
import tqdm
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_val_score

from numpy import mean
from numpy import std
import pickle
from os import path
from sklearn.model_selection import cross_val_score
from warnings import simplefilter
from collections import OrderedDict
from sklearn.metrics import accuracy_score, auc, confusion_matrix, balanced_accuracy_score, precision_recall_curve, auc, roc_curve, roc_auc_score

from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

Entrez.tool = "Zoonosis predictor"

Entrez.email = input("Enter an email address to use NCBI e-utils: ")

%cd ../utils
import model_utils
import validation_utils
%cd ..

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [2]:
best_gradBoost = pickle.load(open('/models/curr_models/gradBoost.pkl', 'rb'))
best_xgBoost = pickle.load(open('/models/curr_models/xgBoost.pkl', 'rb'))
nardus_model = pickle.load(open('/models/nardus_testing/gradBoost.pkl', 'rb'))
gridsearch = pickle.load(open('/models/gridsearch/gradBoost.pkl', 'rb')) # poorest model

kmer = 4

s = product('acgt',repeat = kmer)
permset = set(["".join(x) for x in list(s)])

preds = []
reals = []
for ind, file in enumerate(os.listdir("data/virome_contigs")):
    fasta_sequences = SeqIO.parse(open(f"data/virome_contigs/{file}"),'fasta')

    fastas = [x for x in fasta_sequences]
    for fasta in fastas:
        name, sequence = fasta.id, str(fasta.seq)
        X_info = sequence.lower()

        oDict = assign_kmers_to_dict(X_info, permset, kmer) # convert ordereddict to pandas dataframe

        kmer_df = pd.DataFrame()

        for i in oDict:
            kmer_df.at[0, i]=oDict[i]

        cols_when_model_builds = best_gradBoost.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        gradboost = best_gradBoost.predict_proba(kmer_df)[:,1]
        
        cols_when_model_builds = best_xgBoost.get_booster().feature_names
        kmer_df=kmer_df[cols_when_model_builds]

        xgpred = best_xgBoost.predict_proba(kmer_df)[:,1]

        cols_when_model_builds = nardus_model.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        narduspred = nardus_model.predict_proba(kmer_df)[:,1]

        cols_when_model_builds = gridsearch.feature_names_in_
        kmer_df=kmer_df[cols_when_model_builds]
        
        gridsearchpred = gridsearch.predict_proba(kmer_df)[:,1]

        preds.append([gradboost, xgpred, narduspred, gridsearchpred])

preds = np.array(preds)